In [1]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

C:\Users\ASDF\anaconda3\envs\gnn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model Construction 

In [2]:
class GCNConv(MessagePassing):
    # Applies a linear tranformation to the incoming data
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Multiply with weights
        x = self.lin(x)

        # Step 3: Calculate the normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4: Propagate the embeddings to the next layer
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x,
                              norm=norm)

    def message(self, x_j, norm):
        # Normalize node features.
        return norm.view(-1, 1) * x_j


In [3]:
class Net(torch.nn.Module):

    def __init__(self, dataset):
        super(Net, self).__init__()
        # two convolution layers of GCN
        # first represnt the input and the second represnt the output layer
        # dataset.num_node_features== number of input
        # 16 hidden layers
        # dataset.num_classes==outputclass
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
    # represent the weights updates
    # weights in output
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
#         print(x)
#         print(edge_index)
        x = self.conv1(x, edge_index)
#         print('first convo X:',x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
#         print('sec convo X:',x)

        return F.log_softmax(x, dim=1)



In [4]:
def plot_dataset(dataset):
    edges_raw = dataset.data.edge_index.numpy()
    edges = [(x, y) for x, y in zip(edges_raw[0, :], edges_raw[1, :])]
    labels = dataset.data.y.numpy()

    G = nx.Graph()
    G.add_nodes_from(list(range(np.max(edges_raw))))
    G.add_edges_from(edges)
    plt.subplot(111)
    options = {
                'node_size': 30,
                'width': 0.2,
    }
    nx.draw(G, with_labels=False, node_color=labels.tolist(), cmap=plt.cm.tab10, font_weight='bold', **options)
    plt.show()


In [5]:
def test(data, train=True):
    model.eval()

    correct = 0
    pred = model(data).max(dim=1)[1]

    if train:
        correct += pred[data.train_mask].eq(data.y[data.train_mask]).sum().item()
        return correct / (len(data.y[data.train_mask]))
    else:
        correct += pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
        return correct / (len(data.y[data.test_mask]))

In [6]:
def train(data, plot=True):
    train_accuracies, test_accuracies = list(), list()
    for epoch in range(1000):
            model.train()
            optimizer.zero_grad()
            out = model(data)
#             print('output by softmax:',out)
#             print('data y for comparison',data.y[data.train_mask])
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
#             print('first loss:',loss)
            loss.backward()
            optimizer.step()

            train_acc = test(data)
            test_acc = test(data, train=False)

            train_accuracies.append(train_acc)
            test_accuracies.append(test_acc)
            print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Test Acc: {:.5f}'.
                  format(epoch, loss, train_acc, test_acc))

    torch.save(model.state_dict(), 'new_model_weights.pth')

In [7]:
# if plot:
#     plt.plot(train_accuracies, label="Train accuracy")
#     plt.plot(test_accuracies, label="Validation accuracy")
#     plt.xlabel("# Epoch")
#     plt.ylabel("Accuracy")
#     plt.legend(loc='upper right')
#     plt.show()

# Dataset loading 

In [8]:
dataset = Planetoid(root='cora_dataset', name='Cora')

In [9]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [10]:
dataset.data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [11]:
len(np.unique(dataset.data.edge_index[0]))
# min(dataset.data.edge_index[1])

2708

In [12]:
dataset.data.y

tensor([3, 4, 4,  ..., 3, 3, 3])

In [13]:
print("train_mask=",dataset.data.train_mask[1])
print("train_mask=",dataset.data.y[1])

train_mask= tensor(True)
train_mask= tensor(4)


In [14]:
print("train_mask all=",dataset.data.train_mask)
print("val_mask all=",dataset.data.val_mask)
print("y=",dataset.data.y)

train_mask all= tensor([ True,  True,  True,  ..., False, False, False])
val_mask all= tensor([False, False, False,  ..., False, False, False])
y= tensor([3, 4, 4,  ..., 3, 3, 3])


In [15]:
import torch
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [16]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [17]:
print(dataset)
print("number of graph:" ,len(dataset))
print("number of classes:", dataset.num_classes)
print("number of node features:", dataset.num_node_features)
# print("number of edge features:\t",dataset.num_edge_features)

Cora()
number of graph: 1
number of classes: 7
number of node features: 1433


In [18]:
print("edge_index shape:\t", dataset.data.edge_index.shape)
print("edge_index=",dataset.data.edge_index)
print('\n')
print("train_mask shape:\t", dataset.data.train_mask.shape)
print("train_mask=",dataset.data.train_mask)
print('\n')
print("x shape:\t", dataset.data.x.shape)
print("Shape of x=",dataset.data.x)
print('\n')
print("y shape:\t", dataset.data.y.shape)
print('y',dataset.data.y)

edge_index shape:	 torch.Size([2, 10556])
edge_index= tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


train_mask shape:	 torch.Size([2708])
train_mask= tensor([ True,  True,  True,  ..., False, False, False])


x shape:	 torch.Size([2708, 1433])
Shape of x= tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y shape:	 torch.Size([2708])
y tensor([3, 4, 4,  ..., 3, 3, 3])


In [19]:
# print("Total data view:",data)

In [20]:
# for i in data:
#     print(i)

In [21]:
model = Net(dataset)

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Train model function call 

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [24]:
torch.cuda.is_available()

True

In [25]:
data = dataset[0]

In [26]:
train(data, plot=True)

Epoch: 000, Loss: 1.95180, Train Acc: 0.32143, Test Acc: 0.16200
Epoch: 001, Loss: 1.89694, Train Acc: 0.70000, Test Acc: 0.50400
Epoch: 002, Loss: 1.81418, Train Acc: 0.75714, Test Acc: 0.62500
Epoch: 003, Loss: 1.71816, Train Acc: 0.82143, Test Acc: 0.64200
Epoch: 004, Loss: 1.63399, Train Acc: 0.85000, Test Acc: 0.64200
Epoch: 005, Loss: 1.52087, Train Acc: 0.89286, Test Acc: 0.66900
Epoch: 006, Loss: 1.41496, Train Acc: 0.90000, Test Acc: 0.69300
Epoch: 007, Loss: 1.32301, Train Acc: 0.90714, Test Acc: 0.71500
Epoch: 008, Loss: 1.23021, Train Acc: 0.93571, Test Acc: 0.73200
Epoch: 009, Loss: 1.12306, Train Acc: 0.94286, Test Acc: 0.75000
Epoch: 010, Loss: 1.05656, Train Acc: 0.95714, Test Acc: 0.76700
Epoch: 011, Loss: 0.94277, Train Acc: 0.95714, Test Acc: 0.77300
Epoch: 012, Loss: 0.82854, Train Acc: 0.97143, Test Acc: 0.78400
Epoch: 013, Loss: 0.74750, Train Acc: 0.97143, Test Acc: 0.78500
Epoch: 014, Loss: 0.64894, Train Acc: 0.97857, Test Acc: 0.78000
Epoch: 015, Loss: 0.60139

Epoch: 127, Loss: 0.04205, Train Acc: 1.00000, Test Acc: 0.81500
Epoch: 128, Loss: 0.04735, Train Acc: 1.00000, Test Acc: 0.81500
Epoch: 129, Loss: 0.03725, Train Acc: 1.00000, Test Acc: 0.81300
Epoch: 130, Loss: 0.04261, Train Acc: 1.00000, Test Acc: 0.81300
Epoch: 131, Loss: 0.03274, Train Acc: 1.00000, Test Acc: 0.81300
Epoch: 132, Loss: 0.05073, Train Acc: 1.00000, Test Acc: 0.81200
Epoch: 133, Loss: 0.03196, Train Acc: 1.00000, Test Acc: 0.81400
Epoch: 134, Loss: 0.04885, Train Acc: 1.00000, Test Acc: 0.81400
Epoch: 135, Loss: 0.03063, Train Acc: 1.00000, Test Acc: 0.81200
Epoch: 136, Loss: 0.05745, Train Acc: 1.00000, Test Acc: 0.81200
Epoch: 137, Loss: 0.04391, Train Acc: 1.00000, Test Acc: 0.81100
Epoch: 138, Loss: 0.03363, Train Acc: 1.00000, Test Acc: 0.81100
Epoch: 139, Loss: 0.04967, Train Acc: 1.00000, Test Acc: 0.80900
Epoch: 140, Loss: 0.03712, Train Acc: 1.00000, Test Acc: 0.80900
Epoch: 141, Loss: 0.04052, Train Acc: 1.00000, Test Acc: 0.80700
Epoch: 142, Loss: 0.03034

Epoch: 255, Loss: 0.01888, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 256, Loss: 0.03822, Train Acc: 1.00000, Test Acc: 0.80200
Epoch: 257, Loss: 0.03327, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 258, Loss: 0.03215, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 259, Loss: 0.02739, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 260, Loss: 0.02882, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 261, Loss: 0.01986, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 262, Loss: 0.03011, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 263, Loss: 0.02016, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 264, Loss: 0.03732, Train Acc: 1.00000, Test Acc: 0.80200
Epoch: 265, Loss: 0.02838, Train Acc: 1.00000, Test Acc: 0.80000
Epoch: 266, Loss: 0.02091, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 267, Loss: 0.02556, Train Acc: 1.00000, Test Acc: 0.80200
Epoch: 268, Loss: 0.02248, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 269, Loss: 0.01343, Train Acc: 1.00000, Test Acc: 0.80700
Epoch: 270, Loss: 0.03650

Epoch: 382, Loss: 0.03114, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 383, Loss: 0.02000, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 384, Loss: 0.02261, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 385, Loss: 0.03682, Train Acc: 1.00000, Test Acc: 0.79600
Epoch: 386, Loss: 0.02820, Train Acc: 1.00000, Test Acc: 0.79700
Epoch: 387, Loss: 0.02526, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 388, Loss: 0.02305, Train Acc: 1.00000, Test Acc: 0.80100
Epoch: 389, Loss: 0.02435, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 390, Loss: 0.02527, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 391, Loss: 0.01681, Train Acc: 1.00000, Test Acc: 0.81000
Epoch: 392, Loss: 0.03719, Train Acc: 1.00000, Test Acc: 0.81300
Epoch: 393, Loss: 0.03229, Train Acc: 1.00000, Test Acc: 0.81000
Epoch: 394, Loss: 0.02926, Train Acc: 1.00000, Test Acc: 0.80900
Epoch: 395, Loss: 0.01782, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 396, Loss: 0.02084, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 397, Loss: 0.02048

Epoch: 526, Loss: 0.02086, Train Acc: 1.00000, Test Acc: 0.79800
Epoch: 527, Loss: 0.02309, Train Acc: 1.00000, Test Acc: 0.79600
Epoch: 528, Loss: 0.01347, Train Acc: 1.00000, Test Acc: 0.79800
Epoch: 529, Loss: 0.02689, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 530, Loss: 0.01303, Train Acc: 1.00000, Test Acc: 0.80200
Epoch: 531, Loss: 0.02933, Train Acc: 1.00000, Test Acc: 0.80100
Epoch: 532, Loss: 0.02534, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 533, Loss: 0.02530, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 534, Loss: 0.01432, Train Acc: 1.00000, Test Acc: 0.80700
Epoch: 535, Loss: 0.01780, Train Acc: 1.00000, Test Acc: 0.80900
Epoch: 536, Loss: 0.03062, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 537, Loss: 0.02426, Train Acc: 1.00000, Test Acc: 0.81000
Epoch: 538, Loss: 0.01587, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 539, Loss: 0.02795, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 540, Loss: 0.02506, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 541, Loss: 0.02074

Epoch: 667, Loss: 0.02139, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 668, Loss: 0.01757, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 669, Loss: 0.02021, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 670, Loss: 0.01827, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 671, Loss: 0.01899, Train Acc: 1.00000, Test Acc: 0.79800
Epoch: 672, Loss: 0.01756, Train Acc: 1.00000, Test Acc: 0.79800
Epoch: 673, Loss: 0.02266, Train Acc: 1.00000, Test Acc: 0.80200
Epoch: 674, Loss: 0.02921, Train Acc: 1.00000, Test Acc: 0.80000
Epoch: 675, Loss: 0.01768, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 676, Loss: 0.01242, Train Acc: 1.00000, Test Acc: 0.79700
Epoch: 677, Loss: 0.03437, Train Acc: 1.00000, Test Acc: 0.79700
Epoch: 678, Loss: 0.01808, Train Acc: 1.00000, Test Acc: 0.79400
Epoch: 679, Loss: 0.01746, Train Acc: 1.00000, Test Acc: 0.79700
Epoch: 680, Loss: 0.01806, Train Acc: 1.00000, Test Acc: 0.79800
Epoch: 681, Loss: 0.01638, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 682, Loss: 0.02323

Epoch: 804, Loss: 0.02593, Train Acc: 1.00000, Test Acc: 0.80700
Epoch: 805, Loss: 0.02010, Train Acc: 1.00000, Test Acc: 0.80700
Epoch: 806, Loss: 0.01191, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 807, Loss: 0.01355, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 808, Loss: 0.01295, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 809, Loss: 0.02649, Train Acc: 1.00000, Test Acc: 0.80400
Epoch: 810, Loss: 0.01832, Train Acc: 1.00000, Test Acc: 0.79800
Epoch: 811, Loss: 0.01943, Train Acc: 1.00000, Test Acc: 0.80000
Epoch: 812, Loss: 0.02049, Train Acc: 1.00000, Test Acc: 0.80000
Epoch: 813, Loss: 0.02152, Train Acc: 1.00000, Test Acc: 0.80100
Epoch: 814, Loss: 0.01625, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 815, Loss: 0.00966, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 816, Loss: 0.02256, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 817, Loss: 0.03494, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 818, Loss: 0.03634, Train Acc: 1.00000, Test Acc: 0.79700
Epoch: 819, Loss: 0.01769

Epoch: 944, Loss: 0.00969, Train Acc: 1.00000, Test Acc: 0.80000
Epoch: 945, Loss: 0.02486, Train Acc: 1.00000, Test Acc: 0.80000
Epoch: 946, Loss: 0.02675, Train Acc: 1.00000, Test Acc: 0.80100
Epoch: 947, Loss: 0.01558, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 948, Loss: 0.02065, Train Acc: 1.00000, Test Acc: 0.80000
Epoch: 949, Loss: 0.01907, Train Acc: 1.00000, Test Acc: 0.79900
Epoch: 950, Loss: 0.01429, Train Acc: 1.00000, Test Acc: 0.80200
Epoch: 951, Loss: 0.02490, Train Acc: 1.00000, Test Acc: 0.80200
Epoch: 952, Loss: 0.01851, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 953, Loss: 0.01785, Train Acc: 1.00000, Test Acc: 0.80100
Epoch: 954, Loss: 0.02723, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 955, Loss: 0.01273, Train Acc: 1.00000, Test Acc: 0.80500
Epoch: 956, Loss: 0.00958, Train Acc: 1.00000, Test Acc: 0.80600
Epoch: 957, Loss: 0.02590, Train Acc: 1.00000, Test Acc: 0.80700
Epoch: 958, Loss: 0.03823, Train Acc: 1.00000, Test Acc: 0.80300
Epoch: 959, Loss: 0.01917

# Prediction on direct model learning

In [27]:
y_pred=model(dataset[0])
y_pred

tensor([[-9.0337e+00, -9.2491e+00, -8.7194e+00,  ..., -7.7566e+00,
         -1.2279e+01, -8.6971e+00],
        [-1.1720e+01, -1.3307e+01, -1.3693e+01,  ..., -2.0504e-05,
         -1.3389e+01, -1.2926e+01],
        [-9.9485e+00, -1.1504e+01, -1.1979e+01,  ..., -4.3085e-04,
         -1.2493e+01, -1.1515e+01],
        ...,
        [-4.0998e-01, -2.2281e+00, -8.6851e+00,  ..., -7.9763e+00,
         -1.5863e+00, -4.1795e+00],
        [-8.6448e+00, -7.3070e+00, -8.2446e+00,  ..., -6.5390e+00,
         -1.1394e+01, -1.0560e+01],
        [-7.9892e+00, -6.6659e+00, -7.8114e+00,  ..., -6.5717e+00,
         -1.0358e+01, -9.9763e+00]], grad_fn=<LogSoftmaxBackward0>)

In [33]:
rs=0
for i in y_pred.tolist():
    rs[i]=np.argmax(i)
    

TypeError: 'int' object does not support item assignment

In [30]:
len(y_pred)

2708

In [37]:
print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
print('actual y values____:',dataset[0].y.tolist())

prediction_by_model: [3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6, 3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5, 3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2, 2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0, 6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5, 5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1, 4, 4, 0, 3, 6, 6, 0, 5, 4, 0, 3, 4, 4, 1, 2, 2, 2, 3, 3, 3, 3, 6, 4, 5, 4, 3, 4, 0, 0, 3, 2, 3, 4, 2, 2, 6, 5, 4, 3, 3, 3, 6, 3, 3, 1, 3, 3, 4, 2, 2, 6, 1, 2, 5, 4, 0, 4, 3, 4, 0, 3, 3, 2, 4, 0, 3, 2, 3, 3, 2, 4, 0, 3, 6, 0, 3, 3, 4, 3, 3, 5, 2, 4, 2, 3, 6, 3, 2, 2, 3, 3, 3, 3, 5, 1, 3, 1, 3, 5, 0, 4, 5, 0, 4, 2, 4, 2, 4, 4, 5, 1, 3, 5, 3, 4, 6, 4, 0, 4, 5, 2, 3, 6, 2, 5, 5, 5, 0, 2, 3, 0, 4, 0, 3, 0, 4, 0, 0, 4, 0, 6, 3, 5, 4, 4, 6, 4, 0, 0, 2, 2, 4, 3, 4, 1, 3, 2, 3, 3, 4, 0, 2, 1, 1, 0, 0, 1, 6, 1, 3, 3, 4, 2, 3, 1, 0, 3, 4, 1, 2, 4, 3, 2, 4, 0, 4, 

In [38]:
print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
print('actual y values____:',dataset[0].y.tolist())

prediction_by_model: [3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6, 3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5, 3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2, 2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0, 6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5, 5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1, 4, 4, 0, 3, 6, 6, 0, 5, 4, 0, 3, 4, 4, 1, 2, 2, 2, 3, 3, 3, 3, 6, 4, 5, 4, 3, 4, 0, 0, 3, 2, 3, 4, 2, 2, 6, 5, 4, 3, 3, 3, 6, 3, 3, 1, 3, 3, 4, 2, 2, 6, 1, 2, 5, 4, 0, 4, 3, 4, 0, 3, 3, 2, 4, 0, 3, 2, 3, 3, 2, 4, 0, 3, 6, 0, 3, 3, 4, 3, 3, 5, 2, 4, 2, 3, 6, 3, 2, 2, 3, 3, 3, 3, 5, 1, 3, 1, 3, 5, 0, 4, 5, 0, 4, 2, 4, 2, 4, 4, 5, 1, 3, 5, 3, 4, 6, 4, 0, 4, 5, 2, 3, 6, 2, 5, 5, 5, 0, 2, 3, 0, 4, 0, 3, 0, 4, 0, 0, 4, 0, 6, 3, 5, 4, 4, 6, 4, 0, 0, 2, 2, 4, 3, 4, 1, 3, 2, 3, 3, 4, 0, 2, 1, 1, 0, 0, 1, 6, 1, 3, 3, 4, 2, 3, 1, 0, 3, 4, 1, 2, 4, 3, 2, 4, 0, 4, 

# Model saving and loading

In [39]:
# saving complete model
torch.save(model, 'Updated_GNN_model_weights.pth')

In [40]:
# load model
model = torch.load('GNN_model_weights.pth')

# Prediction on loaded model

In [ ]:
from torch_geometric.data import Data
print(dataset[0])
import numpy as np
x=np.ones(shape=(1,1433))
xne=dataset.data.x[0:2708]
# print(dataset.data.edge_index.shape)
# xne=xne.unsqueeze(0)
# print(dataset.data.edge_index.shape)
e=dataset.data.edge_index
# e=e.unsqueeze(0)
print(xne.shape)
print(e.shape)

data = Data(x=xne, edge_index=e)
print(data)

y_pred=model(data)
y_pred[0]


In [ ]:
print(type(dataset[0]))

In [33]:
print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
print('actual y values____:',dataset[0].y.tolist())

prediction_by_model: [3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6, 3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5, 3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2, 2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0, 6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5, 5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1, 4, 4, 0, 3, 6, 6, 0, 5, 4, 0, 3, 4, 4, 1, 2, 2, 2, 3, 3, 3, 3, 6, 4, 5, 4, 3, 5, 0, 0, 3, 2, 3, 4, 2, 2, 6, 5, 4, 3, 3, 3, 6, 3, 3, 1, 3, 3, 4, 2, 2, 6, 1, 2, 5, 4, 0, 4, 3, 4, 0, 3, 3, 2, 4, 0, 3, 2, 3, 3, 2, 4, 0, 3, 6, 0, 3, 3, 4, 3, 3, 5, 2, 3, 2, 3, 6, 3, 2, 2, 3, 3, 3, 3, 5, 1, 3, 1, 3, 5, 0, 4, 5, 0, 4, 2, 4, 2, 4, 3, 5, 1, 3, 5, 3, 4, 6, 4, 0, 4, 5, 2, 3, 6, 2, 5, 5, 5, 2, 2, 3, 0, 4, 0, 3, 0, 4, 0, 5, 4, 0, 6, 3, 5, 4, 4, 6, 4, 0, 3, 2, 2, 4, 3, 4, 1, 3, 2, 3, 3, 4, 0, 2, 1, 1, 0, 0, 1, 6, 1, 3, 3, 3, 2, 3, 1, 0, 3, 4, 1, 2, 4, 3, 2, 4, 0, 4, 

# Code End here

In [ ]:
dataset[0]

In [ ]:
dataset.data.x[1]

In [ ]:
dataset.data.edge_index

In [ ]:
dataset.data.y

In [ ]:
model()

In [ ]:
model(dataset.data.x[1],dataset.data.edge_index,dataset.data.)

In [ ]:
    dataset = Planetoid(root='/tmp/Cora', name='Cora')

    print(dataset)
    print("number of graph:" ,len(dataset))
    print("number of classes:", dataset.num_classes)
    print("number of node features:", dataset.num_node_features)
    # print("number of edges:",dataset.num_edge_features_features)

    print("edge_index:\t\t", dataset.data.edge_index.shape)
    print(dataset.data.edge_index)
    print("\n")
    print("train_mask:\t\t", dataset.data.train_mask.shape)
    print(dataset.data.train_mask)
    print("\n")
    print("x:\t\t", dataset.data.x.shape)
    print(dataset.data.x)
    print("\n")
    print("y:\t\t", dataset.data.y.shape)
    print(dataset.data.y)

    # print('x values',dataset[0].x)
    sg_x=dataset[0].x.tolist()[1]
    # print("signal record",dataset[0].x.tolist()[1])
    sg_ed = dataset[0].edge_index.tolist()[1]
    # print(sg_ed )
    # print('y values',dataset[0].y)
    # print('complete values',dataset[0])

    # plot_dataset(dataset)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(dataset).to(device)
    data = dataset[0].to(device)
    print("data check",data)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # train(data, plot=True)

    # saving and load with other approch
    # torch.save(model.state_dict(), 'dic_state_save_model.pt')
    # the_model = model()

    # the_model.load_state_dict(torch.load(PATH))
    # ___________
    # saving complete model
    torch.save(model, 'GNN_model_weights.pth')
    # load model
    model = torch.load('GNN_model_weights.pth')
    # print(model.eval())

    # prediction of all datset
    print("test=",dataset[0])
    print("complete dataset: =", dataset)
    
    y_pred=model(dataset[0])
    # print(y_pred)
    print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
    print('actual y values____:',dataset[0].y.tolist())
    print('data_x',data.x)
    print('data_edges',data.edge_index)
    # test_data(x=data.x,edge_index=data.edge_index)
    print('zero......',data.x[0])
    print('last....',data.x[2707])
    # res=model(data.x[])
    # print(res)



    di={
        'predicted':[np.argmax(i) for i in y_pred.tolist()],
        'actual':dataset[0].y,
    }

    #
    # print(model(di))

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

print(dataset)
print("number of graph:" ,len(dataset))
print("number of classes:", dataset.num_classes)
print("number of node features:", dataset.num_node_features)
# print("number of edges:",dataset.num_edge_features_features)

#     print("edge_index:\t\t", dataset.data.edge_index.shape)
#     print(dataset.data.edge_index)
#     print("\n")
#     print("train_mask:\t\t", dataset.data.train_mask.shape)
#     print(dataset.data.train_mask)
#     print("\n")
#     print("x:\t\t", dataset.data.x.shape)
#     print(dataset.data.x)
#     print("\n")
#     print("y:\t\t", dataset.data.y.shape)
#     print(dataset.data.y)

    # print('x values',dataset[0].x)
sg_x=dataset[0].x.tolist()[1]
    # print("signal record",dataset[0].x.tolist()[1])
sg_ed = dataset[0].edge_index.tolist()[1]
    # print(sg_ed )
    # print('y values',dataset[0].y)
    # print('complete values',dataset[0])

    # plot_dataset(dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset).to(device)
data = dataset[0].to(device)
print("data check",data)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # train(data, plot=True)

    # saving and load with other approch
    # torch.save(model.state_dict(), 'dic_state_save_model.pt')
    # the_model = model()

    # the_model.load_state_dict(torch.load(PATH))
    # ___________
    # saving complete model
torch.save(model, 'GNN_model_weights.pth')
    # load model
model = torch.load('GNN_model_weights.pth')
    # print(model.eval())

    # prediction of all datset
print("test=",dataset[0])
print("complete dataset: =", dataset)
    
y_pred=model(dataset[0])
    # print(y_pred)
# print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
# print('actual y values____:',dataset[0].y.tolist())
# print('data_x',data.x)
#     print('data_edges',data.edge_index)
#     # test_data(x=data.x,edge_index=data.edge_index)
#     print('zero......',data.x[0])
#     print('last....',data.x[2707])
#     # res=model(data.x[])
    # print(res)



#     di={
#         'predicted':[np.argmax(i) for i in y_pred.tolist()],
#         'actual':dataset[0].y,
#     }

    #
    # print(model(di))

In [ ]:
# data.x.reshape(1,1433)

In [ ]:
# y_pred=model(data.x)
# y_pred

In [ ]:
print('zero......:',data.x[0])
# print('last......',data.x[2700])
print('y.........:',data.y[0])
print('mask_train zero:',data.train_mask[0])
print('val_mask zero:',data.val_mask[0])
print('test_mask zero:',data.test_mask[0])
print('total_dataset',dataset[0])

In [ ]:
# data.x[0].tolist()

In [ ]:
data.edge_index

In [ ]:
# y_pred=model(data.x[0],data.edge_index,data.y[0])
# y_pred=model(data.x[0],data.edge_index,data.val_mask[0],data.test_mask[0])

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

    print(dataset)
    print("number of graph:" ,len(dataset))
    print("number of classes:", dataset.num_classes)
    print("number of node features:", dataset.num_node_features)
    # print("number of edges:",dataset.num_edge_features_features)

    print("edge_index:\t\t", dataset.data.edge_index.shape)
    print(dataset.data.edge_index)
    print("\n")
    print("train_mask:\t\t", dataset.data.train_mask.shape)
    print(dataset.data.train_mask)
    print("\n")
    print("x:\t\t", dataset.data.x.shape)
    print(dataset.data.x)
    print("\n")
    print("y:\t\t", dataset.data.y.shape)
    print(dataset.data.y)

    # print('x values',dataset[0].x)
    sg_x=dataset[0].x.tolist()[1]
    # print("signal record",dataset[0].x.tolist()[1])
    sg_ed = dataset[0].edge_index.tolist()[1]
    # print(sg_ed )
    # print('y values',dataset[0].y)
    # print('complete values',dataset[0])

    # plot_dataset(dataset)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(dataset).to(device)
    data = dataset[0].to(device)
    print("data check",data)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # train(data, plot=True)

    # saving and load with other approch
    # torch.save(model.state_dict(), 'dic_state_save_model.pt')
    # the_model = model()

    # the_model.load_state_dict(torch.load(PATH))
    # ___________
    # saving complete model
    torch.save(model, 'GNN_model_weights.pth')
    # load model
    model = torch.load('GNN_model_weights.pth')
    # print(model.eval())

    # prediction of all datset
    print("test=",dataset[0])
    print("complete dataset: =", dataset)
    
    y_pred=model(dataset[0])
    # print(y_pred)
    print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
    print('actual y values____:',dataset[0].y.tolist())
    print('data_x',data.x)
    print('data_edges',data.edge_index)
    # test_data(x=data.x,edge_index=data.edge_index)
    print('zero......',data.x[0])
    print('last....',data.x[2707])
    # res=model(data.x[])
    # print(res)



    di={
        'predicted':[np.argmax(i) for i in y_pred.tolist()],
        'actual':dataset[0].y,
    }

    #
    # print(model(di))

In [ ]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import ChebConv, GCNConv  # noqa

parser = argparse.ArgumentParser()
parser.add_argument('--use_gdc', action='store_true',
                    help='Use GDC preprocessing.')
args = parser.parse_args()

dataset = 'Cora'
path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

if args.use_gdc:
    gdc = T.GDC(self_loop_weight=1, normalization_in='sym',
                normalization_out='col',
                diffusion_kwargs=dict(method='ppr', alpha=0.05),
                sparsification_kwargs=dict(method='topk', k=128,
                                           dim=0), exact=True)
    data = gdc(data)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True,
                             normalize=not args.use_gdc)
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True,
                             normalize=not args.use_gdc)
        # self.conv1 = ChebConv(data.num_features, 16, K=2)
        # self.conv2 = ChebConv(16, data.num_features, K=2)

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

###### 